In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


In [12]:
driver = webdriver.Firefox()


In [13]:
groups = []
for i in range(1, 999):
    if len(groups) == 292:
        break

    driver.get("https://cenapet.org/grupos/" + str(i))

    try:
        element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="map"]'))
        )
    except:
        try:
            element = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located(
                    (By.XPATH, '//*[@class="header"]'))
            )
        except:
            continue

    new_group = {}

    header_content = driver.find_element(
        By.XPATH, '/html/body/div/div/div/div/div[1]/div[1]/div[2]/div')

    new_group['Nome'] = header_content.find_element(By.CSS_SELECTOR, 'h2').text

    header_labels = [
        'Universidade',
        'Campus'
    ]

    other_header_info = header_content.find_elements(By.CSS_SELECTOR, 'p')

    for i in range(len(other_header_info)):
        new_group[header_labels[i]] = other_header_info[i].text

    main_data_elements = driver.find_element(By.CLASS_NAME, 'dados')

    labels = []
    for l in main_data_elements.find_elements(By.CLASS_NAME, 'label'):
        labels.append(l.text)

    values = []
    for v in main_data_elements.find_elements(By.CLASS_NAME, 'conteudo'):
        values.append(v.text)

    for k, v in zip(labels, values):
        new_group[k] = v

    labels = []
    values = []
    contact_data_elements = driver.find_element(By.XPATH, '/html/body/div/div/div/div/div[1]/div[2]/div[1]/div')
    splitted_contact_data = contact_data_elements.text.split('Outras mídias')
    
    labels = splitted_contact_data[0].split('\n')[1:]

    i = 1
    for k in range(len(labels)):
        if labels[k] is None or labels[k] == '':
            labels[k] = 'Contato' + str(i)
            i += 1

    for c in contact_data_elements.find_elements(By.CSS_SELECTOR, 'p'):
        try:
            values.append(c.find_element(By.CSS_SELECTOR, 'a').get_attribute('href'))
            # print(c.find_element(By.CSS_SELECTOR, 'a').get_attribute('href'))
        except:
            pass

    for k, v in zip(labels, values):
        new_group[k] = v
        

    sep_cidade = new_group['Cidade'].split('-')
    new_group['Cidade'] = sep_cidade[0]
    new_group['Estado'] = sep_cidade[-1]
    new_group['Email'] = new_group['Email'].split(':')[-1]

    groups.append(new_group)

df = pd.DataFrame(groups)
df


,Nome,Universidade,Campus,Cidade,Email,Site,Eixo,Temática,Ano de criação,Instagram,Facebook,Twitter,YouTube,Contato1,Estado,Site na instituição
0,PET Bacharelado em Ciência da Computação,Universidade Federal de São Carlos,Campus São Carlos,São Carlos,petbcc@ufscar.br,petbcc.ufscar.br,Ciências Exatas e da Terra,Desenvolvimento de Software,2009,https://instagram.com/petbcc.ufscar,https://fb.com/petbcc,https://twitter.com/petbcc,https://www.youtube.com/channel/UCyoSac6aP0aGW...,https://www.linkedin.com/company/pet-bcc-ufscar,SP,NaN
1,PET Tecnologias em Saúde,Universidade Federal de São Paulo,Campus São Paulo,São Paulo,pet.tecnologias@unifesp.br,pet-tecnologias.blogspot.com/,Interdisciplinar,Estudo das Ciências Exatas aplicadas ao corpo ...,2013,https://instagram.com/pettecnologias,https://www.facebook.com/PET.Tecnologias,NaN,https://www.youtube.com/channel/UCQAbQRCOE0Ygl...,NaN,SP,www.unifesp.br/reitoria/prograd/programas-inst...
2,PET Conexões de Saberes Serviço Social,Universidade Federal de Alagoas,Campus A. C. Simões,Maceió,petssoufal@gmail.com,petssoufal.wixsite.com/petssoufal,Ciências Sociais Aplicadas,Política Nacional de Assistência Social (PNAS),2010,https://instagram.com/petssoufal,NaN,NaN,NaN,http://linktr.ee/petssoufal,AL,ufal.br/estudante/graduacao/programas/educacao...
3,PET ProdBio,Universidade Federal do Espírito Santo,Campus São Mateus,São Mateus,petprodbio@gmail.com,petprodbio.saomateus.ufes.br/,Interdisciplinar,Pesca,2013,https://instagram.com/petprodbio,https://www.facebook.com/PETProdBio/,NaN,https://www.youtube.com/channel/UC3T_Aw60h1Dp0...,https://petprodbio.wixsite.com/petinforma,ES,www.prograd.ufes.br/programa-de-educacao-tutor...
4,GET Engenharia Ambiental e Sanitária,Universidade Federal de Juiz de Fora,Campus Juiz de Fora,Juiz de Fora,get.esa@engenharia.ufjf.br,NaN,Engenharias,NaN,2011,https://instagram.com/get_esa,https://www.facebook.com/getesaufjf,NaN,https://www.youtube.com/channel/UCl2TzKviGXBZp...,https://www.linkedin.com/in/get-esa-ufjf-40111...,MG,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,"PET Conexões de Saberes Tecnologias Sociais, T...",Universidade Federal de Alfenas,Campus Poços de Caldas,Poços de Caldas,petsaberespcaldas@unifal-mg.edu.br,petconexoespcaldas.wixsite.com/conexoesdesaberes,Interdisciplinar,Tecnologias Sociais,2010,https://instagram.com/petsaberes,https://www.facebook.com/pet.conexoes.unifal/,NaN,https://www.youtube.com/channel/UC__wnjnP2o_6x...,NaN,MG,www.unifal-mg.edu.br/graduacao/pet/
288,PET Matemática (Licenciatura),Universidade Federal de Goiás,Campus Samambaia,Goiânia,petmat.ufg@gmail.com,NaN,Ciências Exatas e da Terra,NaN,2007,https://instagram.com/petmatufg,NaN,NaN,NaN,NaN,GO,www.ime.ufg.br/p/37710-programa-de-educacao-tu...
289,PET Enfermagem,Universidade Federal de Goiás,Campus Colemar Natal e Silva,Goiânia,pet.fen.ufg@gmail.com,NaN,Ciências da Saúde,ENFERMAGEM,1996,https://instagram.com/petenfermagemufg,NaN,NaN,https://www.youtube.com/channel/UCP2mGJ174syLk...,NaN,GO,www.fen.ufg.br/p/18207-pet-fen-apresentacao
290,PET Nutrição,Universidade Federal de Goiás,Campus Colemar Natal e Silva,Goiânia,petnutufg@gmail.com,petnutufg.blogspot.com/,Ciências da Saúde,NaN,2007,https://instagram.com/petnutufg,NaN,NaN,https://www.youtube.com/channel/UCJnJ3BsXTcgdB...,NaN,GO,fanut.ufg.br/p/30472-petnut-programa-de-educac...


In [15]:
df.to_csv('./cenapet.csv', index=False)